In [1]:
%pip install faiss-cpu -U sentence-transformers mistralai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu



## Read csv


In [2]:
import pandas as pd

df = pd.read_csv("/content/Box_Office_Manual.csv")
df["combined"] = (
    "Title: " + df['Section/Column'].str.strip() + " " + df['Name'].str.strip() +"; Content: " + df['Notes'].str.strip()
)

df.head()

,Section/Column,Name,Notes,combined
0,Venue Information,Melbourne Town Hall (MTH),"Location: 90/130 Swanston St, Melbourne VIC 30...",Title: Venue Information Melbourne Town Hall (...
1,Venue Information,General Venue Shift Information,\n\nGeneral Info \n \nConcert Sheets \nA docum...,Title: Venue Information General Venue Shift I...
2,Venue Information,Hamer Hall,"Location: Arts Centre Melbourne, 100 St Kilda ...",Title: Venue Information Hamer Hall; Content: ...
3,Venue Information,Swing Arm Seats (Hamer Hall),"In Hamer Hall, Swing Arm seats are available i...",Title: Venue Information Swing Arm Seats (Hame...
4,Venue Information,Robert Blackwood Hall (RBH/Monash),"Location: 49 Scenic Blvd, Clayton VIC 3168 (Mo...",Title: Venue Information Robert Blackwood Hall...


In [3]:
import csv

def extract_text_from_csv(file_path):
    all_text = []
    try:
        with open(file_path, mode='r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            for row in reader:
                all_text.append(" ".join(row)) # Join cells in a row with a space
        return "\n".join(all_text) # Join all rows with a newline
    except FileNotFoundError:
        return "Error: File not found."
    except Exception as e:
        return f"An error occurred: {e}"

all_text = extract_text_from_csv('/content/Box_Office_Manual.csv')

In [4]:
print(all_text[:100])
print(len(all_text))

﻿Section/Column Name Notes
Venue Information Melbourne Town Hall (MTH) Location: 90/130 Swanston St,
192855


## chunking


In [5]:
chunk_size = 5000
chunks = [all_text[i : i + chunk_size] for i in range(0, len(all_text), chunk_size)]
len(chunks)

39

we split documents into smaller chunks so it's more effective to identify and retrieve the most relevant information.


## embed


for each text chunk, we create a text embedding, which are numerical representations of text in the vector space, where words with similar meanings are closer to each other in this space.


In [33]:
import os
hf_token = os.environ.get("HF_TOKEN")

In [6]:
import numpy as np

model_id = "sentence-transformers/all-MiniLM-L6-v2"
#hf_token = "{Token}"


#run embedding model locally
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(model_id)

def embed(text):
    output = model.encode(text)
    return output

embeddings = embed(chunks)

embeddings
dimension = embeddings.shape[1]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [34]:
#call embedding model from Huggingface API
from huggingface_hub import InferenceClient
import os

def embed(text):
    client = InferenceClient(model="sentence-transformers/all-MiniLM-L6-v2", token=hf_token)
    output = client.feature_extraction(text)
    return output

embeddings = embed(chunks)


In [23]:
embeddings

array([[ 0.0544122 , -0.021014  , -0.01923657, ...,  0.06683539,
        -0.09857199,  0.02798876],
       [ 0.10474872,  0.00316236, -0.03096059, ...,  0.00199834,
        -0.13633662,  0.07389923],
       [ 0.00877969, -0.02305214, -0.03892595, ...,  0.04241526,
        -0.03835157, -0.03418703],
       ...,
       [-0.03858824, -0.05054043,  0.01894621, ..., -0.14056973,
        -0.05585265,  0.0111223 ],
       [ 0.03732225, -0.01893063,  0.02906756, ...,  0.03239048,
        -0.04250595,  0.03262194],
       [-0.0087201 , -0.05737019,  0.08090882, ..., -0.00204132,
        -0.09115542, -0.02310321]], dtype=float32)

In [24]:
dimension

384

In [11]:
embeddings_df = pd.DataFrame(embeddings)
embeddings_df.to_csv("embeddings.csv", index=False)

## Storeing embedding vectors for retrieval


once we have the embeddings, we store them in a vector db for efficient processing and retrieval - using faiss here

In [25]:
import faiss

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)

## query


when the user asks a question, it is also embedded using the function from above.


In [26]:
question = "Where is Melbourne Town Hall?"
question_embeddings = np.array([embed(question)])

## retrieval


we perform search on our vector db using `index.search`, it takes two parameters, the embedding of our question and k, which is the number of similar vectors to retrieve

The function returns the distances (D) and indices (I) of the most similar vector, and based on the indices, we can return the actual text.


In [27]:
D, I = index.search(question_embeddings, k=2)  # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

## Create prompt


we create a prompt template that combines the retrieved chunk that contains the most relevant/simalar info and then the users question


In [28]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

## Chat model


We then pass this prompt to the LLM and generate an answer based on the user's question and the context retrieved from our documentation. Here we're using Mistral's 'mistral-medium' model via the Mistral API


In [29]:
from mistralai.client import MistralClient
from mistralai import Mistral

client = Mistral(api_key="API_KEY")

def run_mistral(user_message, model="mistral-medium"):
    messages = [{"role":"user", "content":user_message}]
    chat_response = client.chat.complete(model=model, messages=messages)
    return chat_response.choices[0].message.content


run_mistral(prompt)

'Melbourne Town Hall is located at 90/130 Swanston St, Melbourne VIC 3000.'

## all together now


In [17]:
from faiss import IndexFlatL2

prompt = """
Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query}
Answer:
"""


def ask(query: str, index: IndexFlatL2, chunks):
    embedding = embed(query)
    embedding = np.array([embedding])

    _, indexes = index.search(embedding, k=2)
    context = [chunks[i] for i in indexes.tolist()[0]]

    user_message = prompt.format(context=context, query=query)

    messages = [{"role":"user", "content":user_message}]
    chat_response = client.chat.complete(model="mistral-medium", messages=messages)
    return chat_response.choices[0].message.content


ask("What do i need to bring to Plenary for a venue shift?", index, chunks)

'The context information provided does not mention anything about "Plenary" or what needs to be brought for a venue shift there. The details cover venues like Melbourne Town Hall (MTH), Hamer Hall, and Sidney Myer Music Bowl (SMMB), but there is no reference to a venue named "Plenary."\n\nIf you are referring to a different venue or if "Plenary" is a typo or alternative name for one of the mentioned venues, please clarify or provide additional context. Otherwise, I cannot provide an answer based on the given information.'

In [44]:
ask("How do i add an e-membership?", index, chunks)

'To add an e-membership to a member\'s constituency, follow these steps:\n\n1. **Access the Member\'s Profile**: Log in to the Tessi system and locate the member\'s profile.\n2. **Navigate to the Attributes Tab**: Once in the member\'s profile, go to the "Attributes" tab.\n3. **Select the e-Membership Attribute**: In the Attributes tab, find and select the attribute labeled "eMembership."\n4. **Set the Attribute Value**: Set the attribute value to "Yes."\n5. **Save and Close**: Save the changes and close the profile.\n\nThis process ensures that the member\'s welcome pack and other membership materials are sent digitally. You can verify if a digital membership pack has been sent by checking the "e-membership" attribute in the member\'s Tessi account under the Attributes tab.'

In [45]:
ask("Are there steps in Hamer Hall Stalls?", index, chunks)

'The context information does not explicitly mention the presence of steps in the Stalls section of Hamer Hall. However, it does provide details about the location of Swing Arm Seats in the Stalls section, which are located on the aisle seats in specific rows (B, E, H, L, P, T, and W). This suggests that there are aisles and possibly steps between rows, but it is not explicitly confirmed.\n\nFor more detailed information, you might need to refer to the seat map or additional resources provided in the context, such as the link to the Hamer Hall seat map.'

In [46]:
ask("How do installment plans work?", index, chunks)

'Based on the provided context, here’s how installment plans work:\n\n1. **Payment Schedule**: Installment plans allow customers to split payments over multiple dates. The **Instalment Payment Date Calculator** (linked in the context) is used to determine the payment dates, including any applicable grace periods.\n\n2. **Late Payments**:\n   - If a payment fails, a **$10 late fee** is added.\n   - Customers are contacted to correct the payment error by the following Thursday to avoid subscription cancellation.\n   - If no response is received by the following Friday, the subscription may be canceled, with a refund issued minus fees (including the $10 late fee).\n\n3. **Flexibility for Missed Payments**:\n   - Changes to installment payment dates can be made if individual circumstances prevent payment for a specific month.\n   - Only the failed payment date can be adjusted; subsequent scheduled payments remain unchanged.\n   - If a payment date is changed, **Nicole** must be notified to